Mapping particle tracks from Ocean Parcels unto the Salish Sea Atlantis Boxes. 
Original code written by Bec Gordon & Javier Porobic, CSIRO.
Link to the [SSAM Ocean Parcels Repo](https://bitbucket.csiro.au/users/por07g/repos/ssam_oceanparcels/browse)

In [1]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint

In [2]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df_original = gpd.read_file(shapefile_name)
data_df_original = data_df_original.sort_values(by=['BOX_ID'])
data_df = data_df_original.set_index('BOX_ID')
box_depth = data_df['BOTZ']
box_area = data_df['AREA']

In [3]:
surface_layer = []
for box in range(0,130):
    if data_df.iloc[box].BOTZ < 26:
        layer = 0
    elif data_df.iloc[box].BOTZ == 50:
        layer = 1
    elif data_df.iloc[box].BOTZ == 100:
        layer = 2
    elif data_df.iloc[box].BOTZ == 200:
        layer = 3
    elif data_df.iloc[box].BOTZ > 200 and box_depth[box] < 401:
        layer = 4
    elif data_df.iloc[box].BOTZ > 400:
        layer = 5
    surface_layer.append(layer)

In [4]:
len(surface_layer)

130

In [25]:
# Ocean Parcels Spill File

inputFileName = '7a_JStrait_BunkerC_2020-01-03'
num_particles = 10000
file_path = 'results/' + inputFileName + '_'+ str(num_particles) +'_OP_D50_wp3.zarr'

In [26]:
scenario = inputFileName.split(sep = '_')

In [27]:
# Oil type properties & spill location selection

Dilbit = {
    "Density": 1011.2, #kg/m^3
    "Naphthalene": 24, #mg/kg oil
    "Phenanthrene": 17,
    "Pyrene": 10,
    "Benzo": 3,
}

BunkerC = {
    "Density": 995.3,
    "Naphthalene": 680,
    "Phenanthrene": 796,
    "Pyrene": 266,
    "Benzo": 56,
}

Diesel = {
    "Density": 831.0,
    "Naphthalene": 3664,
    "Phenanthrene": 1000,
    "Pyrene": 0.000,
    "Benzo": 0.000,
}

Crude = {
    "Density": 884.7,
    "Naphthalene": 654,
    "Phenanthrene": 327,
    "Pyrene": 13,
    "Benzo": 2,
}

fuel_type = {
    "Dilbit" : Dilbit,
    "BunkerC" : BunkerC,
    "Diesel" : Diesel,
    "Crude" : Crude,
}

spill_volume = {
    "5b" : 2000, #m^3 
    "6a" : 15,
    "7a" : 1000,
    "4a" : 500,
}

spill_box_surface_volume = {
    "5b" : (322271112.331102*25), #m^3 area x surface depth
    "6a" : (108463283.03614*25),
    "7a" : (663754967.760742*25),
    "4a" : ((289374380+143789739)/2*25),
}

In [28]:
# Calculations of oil per particle in mg/m^3/particle
release_start = scenario[3]
oil_per_particle = (fuel_type[scenario[2]]["Density"] * spill_volume[scenario[0]] / spill_box_surface_volume[scenario[0]]) / num_particles #kg/m3
naph_per_particle = oil_per_particle * fuel_type[scenario[2]]["Naphthalene"] 
phen_per_particle = oil_per_particle * fuel_type[scenario[2]]["Phenanthrene"]
pyrene_per_particle = oil_per_particle * fuel_type[scenario[2]]["Pyrene"]
benzo_per_particle = oil_per_particle * fuel_type[scenario[2]]["Benzo"]
release_start_time = np.datetime64(release_start)

In [29]:
# Calculations of oil mass in mg
oil_mass_kg = (fuel_type[scenario[2]]["Density"] * spill_volume[scenario[0]])
naph_mass_mg = oil_mass_kg * fuel_type[scenario[2]]["Naphthalene"]
phen_mass_mg = oil_mass_kg * fuel_type[scenario[2]]["Phenanthrene"]
pyrene_mass_mg = oil_mass_kg * fuel_type[scenario[2]]["Pyrene"]
benzo_mass_mg = oil_mass_kg * fuel_type[scenario[2]]["Benzo"]
oil_mass_mg = oil_mass_kg * 1e6

print(str(scenario[2])+' spill mass of '+str(oil_mass_mg)+' in mg')
print('Naphthalene spill mass of '+str(naph_mass_mg)+' in mg')
print('Phenanthrene spill mass of '+str(phen_mass_mg)+' in mg')
print('Pyrene spill mass of '+str(pyrene_mass_mg)+' in mg')
print('Benzo spill mass of '+str(benzo_mass_mg)+' in mg')

BunkerC spill mass of 995300000000.0 in mg
Naphthalene spill mass of 676804000.0 in mg
Phenanthrene spill mass of 792258800.0 in mg
Pyrene spill mass of 264749800.0 in mg
Benzo spill mass of 55736800.0 in mg


In [30]:
# Calculating the surface concentration of each contaminant for the entire spill
oil_full = oil_per_particle * num_particles * 1e6 / spill_box_surface_volume[scenario[0]]
naph_full = naph_per_particle * num_particles
phen_full = phen_per_particle * num_particles
pyrene_full = pyrene_per_particle * num_particles
benzo_full = benzo_per_particle * num_particles

print(str(scenario[2])+' spill concentration of '+str(oil_full)+' in mg/m^3')
print('Naphthalene spill concentration of '+str(naph_full)+' in mg/m^3')
print('Phenanthrene spill concentration of '+str(phen_full)+' in mg/m^3')
print('Pyrene spill concentration of '+str(pyrene_full)+' in mg/m^3')
print('Benzo spill concentration of '+str(benzo_full)+' in mg/m^3')

BunkerC spill concentration of 3.614584797868778e-09 in mg/m^3
Naphthalene spill concentration of 0.040786376471623594 in mg/m^3
Phenanthrene spill concentration of 0.04774405245795938 in mg/m^3
Pyrene spill concentration of 0.01595467079625276 in mg/m^3
Benzo spill concentration of 0.003358878062369002 in mg/m^3


In [31]:
numLayers = 7
numBoxes = data_df.shape[0]
outputDT = 43100.00
print('numBoxes = ' + str(numBoxes))

numBoxes = 130


In [32]:
pfile = xr.open_zarr(file_path) #.to_netcdf(new_file_name)

In [33]:
#an array of times that correspond to Atlantis timesteps
time_slice = np.arange(0,len(pfile.time[0]),12) #or any N

In [34]:
time_slice

array([  0,  12,  24,  36,  48,  60,  72,  84,  96, 108, 120, 132, 144,
       156, 168, 180, 192, 204, 216, 228, 240])

In [35]:
reduced_pfile = pfile.isel(obs=(time_slice))

In [36]:
reduced_pfile

<xarray.Dataset>
Dimensions:      (trajectory: 10029, obs: 21)
Coordinates:
  * obs          (obs) int32 0 12 24 36 48 60 72 ... 168 180 192 204 216 228 240
  * trajectory   (trajectory) int64 105 108 ... -9223372036854775808
Data variables:
    age          (trajectory, obs) float64 dask.array<chunksize=(226, 1), meta=np.ndarray>
    beached      (trajectory, obs) float64 dask.array<chunksize=(226, 1), meta=np.ndarray>
    decay_value  (trajectory, obs) float32 dask.array<chunksize=(226, 1), meta=np.ndarray>
    lat          (trajectory, obs) float64 dask.array<chunksize=(226, 1), meta=np.ndarray>
    lon          (trajectory, obs) float64 dask.array<chunksize=(226, 1), meta=np.ndarray>
    time         (trajectory, obs) datetime64[ns] dask.array<chunksize=(226, 1), meta=np.ndarray>
    z            (trajectory, obs) float64 dask.array<chunksize=(226, 1), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.4.2

In [37]:
# pulling all variables from Parcels
lon = reduced_pfile.variables['lon']
time = reduced_pfile.variables['time']

In [38]:
numParticles = lon.shape[0]
numSteps = len(time[0,:])

print('numParticles = ' + str(numParticles))
print('numSteps = ' + str(numSteps))

numParticles = 10029
numSteps = 21


In [39]:
minDate = np.datetime64(release_start+"T00:30:00")
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

2020-01-03 00:30:00


In [40]:
# Create the netcdf output file

netcdfFileName = "SSAM_Scenario_"+scenario[0]+"_"+ scenario[3] + "_" + str(num_particles) + "_fromzarr.nc"
try:
    os.remove(netcdfFileName)
except:
    pass
ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)
Dataset.set_fill_on(ncfile)

# Dimensions
t = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numBoxes)
z = ncfile.createDimension("z", numLayers)

In [41]:
# Variables
times = ncfile.createVariable("t",np.float64, ("t",))
oil = ncfile.createVariable("oil",np.float64,("t", "b"))
Naphthalene = ncfile.createVariable("Naphthalene",np.float64, ("t", "b", "z"))
Phenanthrene = ncfile.createVariable("Phenanthrene",np.float64,("t", "b", "z"))
Pyrene = ncfile.createVariable("Pyrene",np.float64,("t", "b", "z"))
Benzo = ncfile.createVariable("Benzo",np.float64,("t", "b", "z"))

# Attributes
Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Naphthalene"
Naphthalene.missing_value = 0.0000
Naphthalene.valid_min = 0.0000
Naphthalene.valid_max = 100000000.0

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Phenanthrene"
Phenanthrene.missing_value = 0.0000
Phenanthrene.valid_min = 0.0000
Phenanthrene.valid_max = 100000000.0

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Pyrene"
Pyrene.missing_value = 0.0000
Pyrene.valid_min = 0.0000
Pyrene.valid_max = 100000000.0

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Benzo(a)pyrene"
Benzo.missing_value = 0.0000
Benzo.valid_min = 0.0000
Benzo.valid_max = 100000000.0

oil.units = "kgOil/m^3"
oil.long_name = "Oil"

times.units = "seconds since " + d
times.dt = outputDT
times.long_name = "time"

In [42]:
# Populate variables with contaminant data

timeData = np.arange(0,(len(time[0,:]))*outputDT,outputDT)
times[:] = timeData

No_layer_particles = np.zeros((numSteps, numBoxes))
Surface_particles = np.zeros((numSteps, numBoxes, numLayers))

In [43]:
for timeValue in range(0, numSteps):

    lats = reduced_pfile['lat'][:,timeValue].values
    lons = reduced_pfile['lon'][:,timeValue].values
    points = np.column_stack((lons, lats))
    multi_point = MultiPoint(points)

    for box_number in range (0, numBoxes):
        layer = surface_layer[box_number]
        box_coordinates = data_df.iloc[box_number].geometry
        found_particles = np.array([box_coordinates.contains(point) for point in multi_point])
        num_particles_within_box = np.sum(found_particles)
        Surface_particles[timeValue][box_number][layer] = Surface_particles[timeValue][box_number][layer] + num_particles_within_box
        No_layer_particles[timeValue][box_number] = No_layer_particles[timeValue][box_number] + num_particles_within_box

oil[:,:] = No_layer_particles * oil_per_particle * 1e6
Naphthalene[:,:,:] = Surface_particles * naph_per_particle
Phenanthrene[:,:,:] = Surface_particles * phen_per_particle
Pyrene[:,:,:] = Surface_particles * pyrene_per_particle
Benzo[:,:,:] = Surface_particles * benzo_per_particle

ncfile.close()

In [44]:
np.histogram(No_layer_particles)

(array([2710,    0,    1,    0,    0,    0,   18,    0,    0,    1]),
 array([    0.,  1000.,  2000.,  3000.,  4000.,  5000.,  6000.,  7000.,
         8000.,  9000., 10000.]))